# Model Testing

This notebook loads the trained Demand Model and RL Agent to run specific test cases.

In [1]:
import pandas as pd
import numpy as np
import pickle
from stable_baselines3 import SAC
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
# Load Artifacts
with open('demand_model_lgbm.pkl', 'rb') as f:
    demand_model = pickle.load(f)

with open('label_encoders.pkl', 'rb') as f:
    le_dict = pickle.load(f)
    
price_constraints = pd.read_csv('price_constraints.csv').set_index('Product ID')

# Load RL Agent
try:
    rl_model = SAC.load('sac_pricing_model')
    print("RL Model loaded.")
except:
    print("RL Model not found. Please run 03_rl_pricing.ipynb first.")
    rl_model = None

RL Model loaded.


## Define Test Case Helper

In [6]:
def run_test_case(product_id, store_id, category, region, price, inventory, 
                  discount, weather, promotion, competitor_price, seasonality, epidemic, date):
    
    # Create DataFrame from input
    data = {
        'Store ID': [store_id],
        'Product ID': [product_id],
        'Category': [category],
        'Region': [region],
        'Inventory Level': [inventory],
        'Price': [price],
        'Discount': [discount],
        'Weather Condition': [weather],
        'Promotion': [promotion],
        'Competitor Pricing': [competitor_price],
        'Seasonality': [seasonality],
        'Epidemic': [epidemic],
        'Date': [pd.to_datetime(date)]
    }
    test_df = pd.DataFrame(data)
    
    # Feature Engineering
    test_df['Month'] = test_df['Date'].dt.month
    test_df['Day'] = test_df['Date'].dt.day
    test_df['Weekday'] = test_df['Date'].dt.weekday
    
    # Encode
    for col, le in le_dict.items():
        if col in test_df.columns:
            val = test_df[col].iloc[0]
            if val in le.classes_:
                test_df[col] = le.transform([val])
            else:
                print(f"Warning: Unseen label {val} for {col}")
                test_df[col] = -1 # Handle unseen
                
    # 1. Predict Demand
    # Ensure correct column order
    # We need to match the features used in training
    features = ['Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level', 
                'Price', 'Discount', 'Weather Condition', 'Promotion', 
                'Competitor Pricing', 'Seasonality', 'Epidemic', 'Month', 'Day', 'Weekday']
    
    demand_pred = demand_model.predict(test_df[features])[0]
    print(f"Predicted Demand (at Price {price}): {demand_pred:.2f}")
    
    # 2. Recommend Price (RL Agent)
    if rl_model:
        # Construct State (features excluding Price) and append previous price to match env obs shape
        feature_cols = [c for c in features if c != 'Price']
        obs_feats = test_df[feature_cols].iloc[0].values.astype(np.float32)
        prev_price = np.array([np.float32(price)])
        obs = np.concatenate([obs_feats, prev_price])  # shape (15,)
        
        # Predict action; pass a single observation (1D)
        action, _ = rl_model.predict(obs, deterministic=True)
        
        # Decode Price Constraints
        try:
            min_price = price_constraints.loc[product_id, 'min_price']
            max_price = price_constraints.loc[product_id, 'max_price']
        except KeyError:
            min_price, max_price = 10, 1000
            
        # Scale Action to Price
        recommended_price = min_price + (max_price - min_price) * (action[0] + 1) / 2
        print(f"Recommended Price (RL): {recommended_price:.2f} (Bounds: {min_price}-{max_price})")
        
        # Predict Demand at Recommended Price
        test_df['Price'] = recommended_price
        opt_demand = demand_model.predict(test_df[features])[0]
        print(f"Predicted Demand at Recommended Price: {opt_demand:.2f}")
        print(f"Estimated Revenue: {recommended_price * opt_demand:.2f}")

## Run Test Cases

In [7]:
# Example Test Case 1
run_test_case(
    product_id='P0001', 
    store_id='S001', 
    category='Electronics', 
    region='North', 
    price=80.0, 
    inventory=50, 
    discount=5, 
    weather='Sunny', 
    promotion=0, 
    competitor_price=75.0, 
    seasonality='Summer', 
    epidemic=0, 
    date='2023-06-15'
)

Predicted Demand (at Price 80.0): 130.67
Recommended Price (RL): 66.81 (Bounds: 7.3-121.94)
Predicted Demand at Recommended Price: 90.40
Estimated Revenue: 6039.53


In [8]:
# Example Test Case 2 (High Seasonality)
run_test_case(
    product_id='P0002', 
    store_id='S002', 
    category='Clothing', 
    region='East', 
    price=120.0, 
    inventory=100, 
    discount=10, 
    weather='Snowy', 
    promotion=1, 
    competitor_price=110.0, 
    seasonality='Winter', 
    epidemic=0, 
    date='2023-12-20'
)

Predicted Demand (at Price 120.0): 270.24
Recommended Price (RL): 57.35 (Bounds: 11.59-99.74)
Predicted Demand at Recommended Price: 78.98
Estimated Revenue: 4529.19
